### Import Libraries

In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import numpy as np
# Suppress warnings
warnings.filterwarnings('ignore')


### Load and Prepare the Data

In [2]:
# Load the dataset 
df = pd.read_csv('data/oversampled_movies_2015_2023_genres.csv')
df.head()

,title,release_year,language,genre,overview,vote_average,vote_count,popularity,cleaned_overview
0,Limbo,2020,Amerikanisch,Humor,offbeat observation refugee waiting granted as...,6.881,105.0,6.726,offbeat observation refugee waiting granted as...
1,"Fear, Inc.",2016,Amerikanisch,Horror,horror junkie joe foster get live ultimate sca...,5.655,187.0,11.771,horror junkie joe foster get live ultimate sca...
2,Dear David,2023,Amerikanisch,Horror,shortly comic artist adam responds internet tr...,4.588,57.0,14.591,shortly comic artist adam responds internet tr...
3,The 5th Wave,2016,Amerikanisch,Action,16yearold cassie sullivan try survive world de...,5.943,5692.0,26.711,yearold cassie sullivan try survive world deva...
4,The Scientist,2020,Amerikanisch,Horror,unconventional scientist struggling care termi...,6.046,108.0,8.472,unconventional scientist struggling care termi...


In [3]:
# Remove lines that contain NAN-Values
df_cleaned = df.dropna()

# Check whether there are still missing values
print(df_cleaned.isnull().sum())


title               0
release_year        0
language            0
genre               0
overview            0
vote_average        0
vote_count          0
popularity          0
cleaned_overview    0
dtype: int64


In [4]:
# Define the features 
X = df_cleaned['cleaned_overview'] + ' ' + df_cleaned['genre'] 
# Predict the language
y = df_cleaned['language']  

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Feature Engineering

In [5]:
# Apply TF-IDF Vectorizer to the combined text features
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [ ]:
# Initialize Logistic Regression with class balancing and parameter tuning
log_reg = LogisticRegression(max_iter=500)

# Define the parameter grid for Logistic Regression
param_grid_reduced = [
    {'penalty': ['l1', 'l2'], 'C': [1, 10, 100], 'solver': ['liblinear']},
    {'penalty': ['elasticnet'], 'C': [1, 10, 100], 'solver': ['saga'], 'l1_ratio': [0, 0.5, 1]}
]

# Apply Grid Search with Cross-Validation to find the best parameters
grid_search_reduced = GridSearchCV(log_reg, param_grid_reduced, cv=5, verbose=1)
grid_search_reduced.fit(X_train_tfidf, y_train)


Fitting 5 folds for each of 15 candidates, totalling 75 fits


In [ ]:
# Output best parameters found
print("Best parameters found: ", grid_search_reduced.best_params_)

In [ ]:
# Make predictions on the test set
y_pred = grid_search_reduced.predict(X_test_tfidf)

# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


In [ ]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Visualize confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=grid_search_reduced.classes_, 
            yticklabels=grid_search_reduced.classes_)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Generate a classification report
classification_report_result = classification_report(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred, target_names=grid_search_reduced.classes_)

print(classification_report_result)
